![ChinesePod](https://chinesepod.com/instapage/academic_offers/32689996-0-logo-black.png)
# ChinesePod Web Scraper (Library List)

This jupyter notebook is an implementation of a simple web scraper designed to scrape the details of the lessons available on ChinesePod. This script does not download the lesson notes, mp3 files or other materials. The purpose of this project is for me to become familiar with web scraping and how to wrangle the data into a usable format. I chose this as a project because I have been studying Chinese (普通话）for a while and I am interested in subscribing to their service. I have watched many of their freely available videos on Youtube and their material appears to be of very high quality. The reason I am considering subscribing to their service is that I think it may help improve my listening skills and extend my vocabulary.

A few questions for which I would like to find answers:

- How many of levels are there and how many lessons in each level?
- What are their names?
- How many lessons for each level.
- How many video lessons?
- How many audio only lessons?
- How many different teachers are there?
- How often do they produce lessons?
- How long have they been making lessons?
- They haven't always made video lessons, how long have they been doing this?
- What are the most common topics covered in the lessons?
- and so on ....

The scraped data and will studied and visualised and used to answer the above questions in a separate notebook ([ChinesePod_Explore.ipynb](ChinesePod_Explore.ipynb)).



### NOTES:
I have used Selenium with the Chrome browser. You will need to:
- Install [Chrome](https://www.google.com/chrome/)
- Check the version of Chrome, then get the corresponding *chromedriver* from [here](https://sites.google.com/a/chromium.org/chromedriver/downloads).
- Unzip *chromedriver*
- Put it somewhere sensible. Since I am using a Mac I have moved it to `/usr/local/bin` by 
```bash
mv chromedriver /usr/local/bin
```

In [1]:
from selenium import webdriver # Web scraping
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup # Web scraping
import re # regular expressions
import urllib.parse # convert redirect url, e.g., %2F --> /
import os # path handling
import time # force a wait period
import csv # writing a csv file
import wget # downloading files # install with pip
import numpy as np
import pandas as pd
from urllib.error import HTTPError, URLError
import dragonmapper.transcriptions # for detecting pinyin # install with pip
import dragonmapper.hanzi # for detecting hanzi

In [2]:
delay = 60 # In seconds

## Local Folder for Storing Lesson Materials (INPUT REQUIRED)

In [3]:
local_CP_dir = './data'

## URLs 

In [4]:
# ChinesePod URLs
# Homepage 
baseURL = 'https://chinesepod.com'
# Login page 
login_url = "https://chinesepod.com/accounts/signin"
# Library list 
libraryURL = baseURL + '/library/list/'

## Start browser

In [5]:
driver = webdriver.Chrome()

## Login to ChinesePod website (INPUT REQUIRED)

In [6]:
# Login using Selenium
def cp_login(my_email,my_password):
    
    # Load login page
    driver.get(login_url)
    
    # Wait for login buttion to be clickable 
    loginBtn = WebDriverWait(driver,delay).until(EC.element_to_be_clickable((By.CLASS_NAME,'btn')))
    
    # Now log in
    driver.find_element_by_name("email").send_keys(my_email)
    driver.find_element_by_name("password").send_keys(my_password)
    loginBtn.click()
    

In [7]:
# INPUT REQUIRED
cp_login("youAre@thisEmail.com","yourPassword") 

## Load the first page of the library list

In [8]:
driver.get(libraryURL) 

## Define functions to perform the scraping

In [9]:
def find_all_lesson_urls_on_a_page(bs):
    """
        Finds all ChinesePod lesson urls on a library list page which has been
        put into the BeautifulSoup object bs.
        
    """
    
    lesson_urls = []
     
    lessonContainer = bs.find('div',{'class':'table-responsive'})
    lessonContainer = lessonContainer.tbody.find_all('tr')
    
    for lesson in lessonContainer:
        lesson_href = lesson.a['href']
        lesson_urls.append(lesson_href)
    
    return lesson_urls 

In [10]:
#lesson_urls = find_all_lesson_urls_on_a_page(bs)

In [11]:
#print(lesson_urls)

In [12]:
#print(len(lesson_urls))

In [13]:
def get_all_lesson_urls(driver):
    
    """ 
        Gets all lesson urls in the library list using driver, which
        is a Selenium web driver that is already logged into Skritter.
        
    """
    
    driver.get(libraryURL)
    time.sleep(10)
    all_lesson_urls = []

    print('Getting all lesson URLs. This may take a while.\n')
    
    bs = BeautifulSoup(driver.page_source,'html')
    
    # First page 
    lesson_urls = find_all_lesson_urls_on_a_page(bs)
    all_lesson_urls +=  lesson_urls
    
    # How many pages 
    pagination = bs.find('ul',{'class':'pagination'})
    listItems = pagination.find_all('li')
    numOfPages = int(listItems[-2].text) 
    

    # The first page is 'https:// ......  page=0' and the last page is numOfPages-1
    for i in range(1,numOfPages):
        
        page_num = str(i)
        page_url = 'https://chinesepod.com/library/list/fetch?searchType=1&series_id=&bookmark=0&studied=0&orderby=1&page=' 
        page_url += page_num
        
        driver.get(page_url)
        time.sleep(2)
    
        bs = BeautifulSoup(driver.page_source,'html')
    
        lesson_urls = find_all_lesson_urls_on_a_page(bs)
        all_lesson_urls += lesson_urls
    
    return all_lesson_urls

In [14]:
#all_lesson_urls = get_all_lesson_urls(driver)

In [15]:
#print(len(all_lesson_urls))

In [16]:
#print(all_lesson_urls)

In [17]:
def extract_name(bs):
    """
        Extacts the title of the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs.
        
    """
    
    temp = bs.find('h1',{'class':'lesson-page-title'})
    name = temp.find('span',{'itemprop':'name'}).text.strip()
    name = name.replace('/','-') # some have slashes, slashes cause issues with file names
    
    return name

In [18]:
def extract_series(bs): # Used to be level
    """
        Extracts the series of the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs.
        
    """
    
    temp = bs.find('h1',{'class':'lesson-page-title'})
    level = temp.small.text.strip()
    
    return level

In [19]:
def extract_level(bs):
    """
        Extracts the level of the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs. From the sidebar not the top.
        
    """
    
    level = bs.find('span',{'class':'fa fa-bar-chart'}).parent.next_sibling.next_sibling.text
    
    return level

In [20]:
def extract_teachers(bs):
    """
        Extracts the teachers of the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs.
    """
    
    Teachers = bs.find('td',{'itemprop':'name'}).text.strip()
    Teachers = Teachers.replace(u'\xa0', u'').split(',')
    
    return Teachers

In [21]:
def extract_key(bs):
    """
        Extracts lesson key of the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs.
        
    """
    
    key = bs.find('span',{'class':'fa fa-key'}).parent.next_sibling.next_sibling.text
    
    return key

In [22]:
def is_video_lesson(bs):
    """
        Determines if the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs is a video lesson.
        
    """
    # ChinesePod uses wistia for hosting the videos
    match = bs.find(src=re.compile('wistia.com'))
    
    tags = extract_tags(bs)
    
    if match or 'video' in tags or 'Video' in tags:
        bIs_video_lesson = True
    else:
        bIs_video_lesson = False
    
    return bIs_video_lesson

In [23]:
def extract_date(bs):
    """
        Extracts the date that the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs was published.
    
    """
    
    datePublished = bs.find('span',{'class':'fa fa-calendar'}).parent.next_sibling.next_sibling.text
    
    return datePublished

In [24]:
def extract_tags(bs):
    """
        Extracts lesson tags if they are present and returns [''] if they are not
        for the ChinesePod lesson at a url which has been put into the BeautifulSoup
        object bs.
    
    """
    
    try:
        tags = bs.find('span',{'class':'fa fa-lightbulb-o'}).parent.next_sibling.next_sibling.text
        tags = tags.replace('\n','').split(',')
        tags = list(map(str.strip,tags))
    except AttributeError:
        tags = ['']
        
    return tags

In [25]:
def extract_functions(bs):
    """
        Extracts lesson functions if they are present and returns [''] if they are not.
        
    """
    try:
        functions = bs.find('span',{'class':'fa fa-wrench'}).parent.next_sibling.next_sibling.text
        functions = functions.replace('\n','').split(',') # could be more than one?
        functions =  list(map(str.strip,functions))
    except AttributeError:
        functions = ['']
    
    return functions

In [26]:
def extract_img_url(bs):
    """
        Extracts image url of the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs.
    
    """
    
    img_url = bs.find_all('img',{'id':'side-img'})[0]
    img_url = img_url['data-src']
    
    return img_url

In [27]:
def extract_descript(bs):
    """
        Extracts the description of the ChinesePod lesson at a url which has been put 
        into the BeautifulSoup object bs.
    
    """
    
    text = bs.find('div',{'class':'lesson-video-content'}).find_all(text=True)
    
    # If a text element contains any of the black listed items it will not be included in the description
    blackList = ['wistia','unpin video','@']
    
    description = ''
    for t in text:
        t_include = True
        for blItem in blackList:
            if blItem in t:
                t_include = False
        if t_include:       
            description = description + t
            
    # Remove preceding and trailing \n       
    description = description.strip()  
    
    return description

In [28]:
def get_lesson_info(lesson_url,driver,directory,verbose=False):
    """
        Extracts information about the ChinesePod lesson at the url 
        lesson_url and returns it in a dictionary.
    
    """
    
    # Create directory if doesn't already exist
    if not os.path.exists(directory):
        os.mkdir(directory)
        print("Directory: " , directory ,  " created.")
    
    # Create a log file for errors, we are expecting possible TimeoutExceptions
    error_log = open(os.path.join(directory,'error_log.txt'), mode = 'a+')
    
    # We will retry 
    total_number_of_tries = 10
    
    error_count = 0
    
    for j in range(0,total_number_of_tries):
        
            try:
    
                # Get the html
                driver.get(lesson_url)

                #time.sleep(delay)
                fakey = WebDriverWait(driver,delay).until(
                    EC.presence_of_element_located((By.CLASS_NAME,'fa-key')))
                
                # if all ok, then break out of the for loop
                break
            
            except TimeoutException as e:
                
                error_count = j + 1
                
                if error_count < total_number_of_tries:
                    
                    # We will try again, but first lets give their server a short break
                    
                    # Load something else
                    driver.get('https://www.google.com/')
                    
                    # Wait 
                    time.sleep(delay)
                    
                else:
                    
                    error_log.write('TimeoutException Thrown and caught multiple times\n')
                    error_log.write('\n')
                    error_log.write('Error on lesson at: ')
                    error_log.write('\n')
                    error_log.write(lesson_url)
                    error_log.write('\n')
                    error_log.write('error_count: ' + str(error_count))
                    error_log.write('\n')
                    error_log.write('Maximum number of retries reached. ')
                    error_log.write('\n')
                    error_log.write('******************************************************')
                    error_log.write('\n')

    error_log.close()    
    
    bs = BeautifulSoup(driver.page_source,'html')

    # Name/Title of the lesson
    lesson_name = extract_name(bs)    
    
    # Level of the lesson
    lesson_level = extract_level(bs)
    
    # Series, e.g., Qing Wen, Elementary, Vocab Tour, ...
    lesson_series = extract_series(bs)
    
    # Teachers 
    Teachers = extract_teachers(bs)
    
    # Key
    key = extract_key(bs)
    
    # Is it a video lesson?
    video =  is_video_lesson(bs)
    
    # Date published
    datePublished = extract_date(bs)
    
    # Tags
    tags = extract_tags(bs)
    
    # Functions (basically another tag??)
    functions = extract_functions(bs)
    
    # Lesson image url
    img_url = extract_img_url(bs)
    
    # Lesson description
    description = extract_descript(bs)
     
    # OUTPUT:
    #########
    
    # Screen:
    if verbose:
        print('Name: ',lesson_name,'\n')
        print('Lesson url: ',lesson_url,'\n')
        print('Level: ',lesson_level,'\n') # This is lesson level
        print('Series: ',lesson_series,'\n')
        print('Teachers: ',' | '.join(Teachers),'\n')
        print('Key: ',key,'\n')
        print('Video: ', video,'\n')
        print('Date published: ',datePublished,'\n')
        print('Tags: ',' | '.join(tags),'\n')
        print('Functions: ',' | '.join(functions),'\n')
        print('Lesson image url: \n',img_url,'\n')
        print('Description: \n',description,'\n')
    else:
        print(key + '\t' + lesson_level + '\t' + lesson_series + '\t' + lesson_name)
    
    # Return everything as a dictionary
    lesson_data = {'Key': key,
                   'Name': lesson_name,
                   'Level': lesson_level,
                   'Series': lesson_series,
                   'URL': lesson_url,
                   'Teachers':Teachers,
                   'Date': datePublished,
                   'Video': video,
                   'Tags': tags,
                   'Functions': functions,
                   'ImageURL': img_url,
                   'Description': description }
    
    return lesson_data

In [29]:
def get_all_lesson_info(driver,directory):
    """
        Get lesson information for every ChinesePod lesson in the library and return 
        it as a list of dictionaries, a dictionary for each lesson. The dictionaries
        contain lesson data.
    
    """
    
    # List of dictionaries containing lesson data
    cp_lesson_data = []
    
    # Get Library info.
    all_lesson_urls = get_all_lesson_urls(driver)
    
    # Get lesson info for all lessons 
    for lesson_url in all_lesson_urls:
        
        data_dic = get_lesson_info(lesson_url,driver,directory)
                
        # Store lesson info
        cp_lesson_data.append(data_dic)
          
    return cp_lesson_data

## Scrape lesson information (But not the lesson materials)

In [30]:
cp_lesson_dics = get_all_lesson_info(driver,local_CP_dir)

Getting all lesson URLs. This may take a while.

Directory:  ./data  created.
QW0465	Pre-Intermediate	Qing Wen	Five ways of seeing: 看、见、看见、看到、看出
4263	Intermediate	Intermediate	Animal Proverbs: Become a pig to catch a tiger
4423	Upper Intermediate	Upper Intermediate	相见恨晚——老电影
4272	Elementary	Elementary	Character mind map: 客 guest
4271	Pre Intermediate	Pre Intermediate	How To Use Action Complete “le” 了
4266	Newbie	Newbie	Is learning Chinese difficult?
JM0001	Any	Journey into Mandopop	Chinese Love Songs (Part 1)
4426	Upper Intermediate	Upper Intermediate	新年特辑：回首2019，展望2020
4262	Elementary	Elementary	Too Many Questions
4265	Intermediate	Intermediate	Animal Proverbs: Can't get both fish and a bear's paw
4672	Advanced	Advanced	孙子兵法-计篇(节选)
4422	Upper Intermediate	Upper Intermediate	语法课程：“了解”和“理解”
4245	Newbie	Newbie	Lesson 1 - The Dish
4246	Newbie	Newbie	Lesson 2 - Hello Toilet
4247	Newbie	Newbie	Lesson 3 - I’m Just A Car
4248	Newbie	Newbie	Lesson 4 - Man or Machine
4249	Newbie	Newbie	Lesson 5

4145	Advanced	Advanced	Do you drink hot water? 你今天喝热开水了吗？
4146	Intermediate	Intermediate	A letter from George H.W. Bush 老布什的一封信
4147	Media	Media	Obsession with mobile games 沉迷游戏──不能自拔
4124	Newbie	Newbie	66 Enjoyable Characters with Joy #3
4142	Media	Media	Jack Ma’s retirement 功成身退──接任马云：创造者张勇和他所改变的阿里巴巴
4141	Media	Media	Job Hopping 另谋高就──闪辞
4137	Elementary	Elementary	The Key to Closing a Deal
4136	Advanced	Advanced	Winning through stability 稳中求胜──中国经济
4135	Intermediate	Intermediate	Online Shopping 网上购物
4131	Upper Intermediate	Upper Intermediate	Venice flooding 洪水猛兽--威尼斯水灾
4123	Newbie	Newbie	66 Enjoyable Characters with Joy #2
4100	Intermediate	Intermediate	Chinese Drinking Culture
4120	Upper Intermediate	Upper Intermediate	Youth in the Nursing Home 养老院为啥住进了年轻人
4122	Upper Intermediate	Upper Intermediate	疯狂富豪和现实生活 Crazy Rich Asians & Real Life
4121	Newbie	Newbie	66 Enjoyable Characters with Joy #1
4119	Upper Intermediate	Upper Intermediate	物尽其用-绿色能源推动农村绿色发展
QW0463	Any	Qing Wen	Tips on How

AI001	Newbie	Newbie	How to Say Hello
AI000	Newbie	Newbie	Intro to All the Way to Intermediate series
4024	Advanced	Advanced	床头吵床尾和 P1
4022	Upper Intermediate	Upper Intermediate	Home Cooking Livestream
4023	Elementary	Elementary	Learning to Cook in Shanghai
4021	Advanced	Advanced	猛虎与蔷薇 Part 3
4020	Upper Intermediate	Upper Intermediate	Ridiculously Small Clothing!
4019	Intermediate	Intermediate	Is Buying Better Than Renting?
4018	Elementary	Elementary	First This, Then That
4017	Advanced	Advanced	猛虎与蔷薇 Part 2
4016	Upper Intermediate	Upper Intermediate	雨巷 'Rain Alley' Poem
QW0448	Intermediate	Qing Wen	Making Up in Chinese
4015	Elementary	Elementary	The Benefits of Reading ft Mandarin Companion
4014	Advanced	Advanced	猛虎与蔷薇 Part 1
QW0447	Intermediate	Qing Wen	Breaking Up in Chinese
4013	Intermediate	Intermediate	Binge Reading
4012	Elementary	Elementary	What's in Your Handbag?
4011	Advanced	Advanced	黑名单 Blacklisted
4010	Upper Intermediate	Upper Intermediate	Interview Questions
4009	Intermedia

2983	Intermediate	Intermediate	Returning Clothes
2982	Elementary	Elementary	Is He Coming or Not?
2981	Upper Intermediate	Upper Intermediate	Visiting a Colleague in Hospital
2980	Advanced	Advanced	大闸蟹 Chinese Mitten Crab
2979	Intermediate	Intermediate	Gym Memberships Plans
QW0428	Intermediate	Qing Wen	Opinions and Suggestions
2978	Advanced	Advanced	面试
2977	Intermediate	Intermediate	Going to Grandma's to Feast
2976	Elementary	Elementary	Did You Go on a Date with Him?
2975	Advanced	Advanced	Gogoro Electric Scooters
2974	Upper Intermediate	Upper Intermediate	Visiting an Aboriginal Festival
2973	Intermediate	Intermediate	It's Hard to Shed Fat
2972	Elementary	Elementary	I'll Uber to the Party
QW0427	Upper-Intermediate	Qing Wen	Synonyms for 'Trick'
2971	Upper Intermediate	Upper Intermediate	Pokemon Go
2970	Intermediate	Intermediate	What Star Sign Are You?
2969	Elementary	Elementary	When Are You Going?
2968	Advanced	Advanced	但愿人长久: A Song song
2967	Upper Intermediate	Upper Intermediate	Getting

2845	Intermediate	Intermediate	Sprained Ankle
2844	Elementary	Elementary	Can You Teach Me Japanese?
2843	Intermediate	Intermediate	Budget Airlines
2842	Upper Intermediate	Upper Intermediate	Korean Wave
2841	Advanced	Advanced	夕阳产业-Industry in Decline
QW0418	Intermediate	Qing Wen	Let's Do Math In Chinese!
2840	Advanced	Advanced	That NBA Game! 望梅止渴
2839	Intermediate	Intermediate	The Two Child Policy
2838	Elementary	Elementary	Could You Pass The...
2837	Intermediate	Intermediate	Wishing People Well at Chinese New Year
2836	Upper Intermediate	Upper Intermediate	New Year's Eve Family Dinner
2835	Intermediate	Intermediate	CNY Calligraphy Special (video)
2834	Elementary	Elementary	Chinese New Year Market Tour (video)
2833	Advanced	Advanced	相亲 Arranged Blind Dates
2832	Upper Intermediate	Upper Intermediate	Looking After the Environment
2831	Intermediate	Intermediate	Dad's Got a Cold
2830	Elementary	Elementary	Drawing the Winning Ticket
2829	Upper Intermediate	Upper Intermediate	Madonna's Asian 

...

0762	Advanced	Advanced	大智若愚
0761	Intermediate	Intermediate	Getting Your Hair Done
QW0032	Elementary	Qing Wen	Extreme Chinese - 不得了 (bùdéliǎo)
0760	Newbie	Newbie	Bumming a Smoke
0759	Advanced	Advanced	好莱坞大片
0758	Elementary	Elementary	Too Picky
CA0019	Dear Amber	Dear Amber	The Non-Chinese Speaking Tourist and Toilets
0757	Intermediate	Intermediate	Lost Cell Phone
0756	Newbie	Newbie	After You
CM0032	Movie Madness	Movie Madness	#32
0755	Upper Intermediate	Upper Intermediate	Bad Service Restaurant
0754	Elementary	Elementary	Preparing for the Heat
QW0031	Intermediate	Qing Wen	Using Verbs 不出来 , 得出来
0753	Intermediate	Intermediate	Singapore
0752	Media	Media	丈夫拒绝签字手术
0751	Upper Intermediate	Upper Intermediate	The Final Jizhou Pieces
CA0018	Dear Amber	Dear Amber	Trip to the Chinese Doctor and a Special Guest
0750	Elementary	Elementary	Getting Water Delivered
0749	Advanced	Advanced	八卦周刊：窃听门事件
CM0031	Movie Madness	Movie Madness	#31
0748	Newbie	Newbie	Before Noon, After Noon
0747	Intermediate	Interm

CM0017	Movie Madness	Movie Madness	#17
0642	Elementary	Elementary	Yang Jie's Diary: Everyone Is Dating
QW0015	Elementary	Qing Wen	You Can and Will Use 会 (huì)!
0641	Intermediate	Intermediate	Beauty Pageant Registration
0640	Newbie	Newbie	What's tomorrow?
0639	Upper Intermediate	Upper Intermediate	Arrival in Jizhou
CA0002	Dear Amber	Dear Amber	Personal Questions and Bargaining
0638	Advanced	Advanced	相亲活动
0637	Elementary	Elementary	Basic Shapes
0636	Intermediate	Intermediate	Death by Ninja
CM0016	Movie Madness	Movie Madness	#16
0635	Media	Media	世界记录
QW0014	Elementary	Qing Wen	More or less?  多 (duō) and 少 (shǎo)
0634	Newbie	Newbie	Identifying Family Members
0633	Elementary	Elementary	Olympic Training
0632	Advanced	Advanced	上火
CA0001	Dear Amber	Dear Amber	In-Laws and Drinking
0631	Newbie	Newbie	How do you take your coffee?
0630	Upper Intermediate	Upper Intermediate	Calisthenics
CM0015	Movie Madness	Movie Madness	#15
0629	Newbie	Newbie	Too Much to Drink
N0006	News and Features	News and Feat

0499	Media	Media	保姆博客假揭秘
0498	Intermediate	Intermediate	Lili and Zhang Liang 13: A Dad Gives Advice to a Broken-Hearted Son
0497	Newbie	Newbie	Requesting a Glass of Water
0496	Upper Intermediate	Upper Intermediate	Capital Punishment
0495	Advanced	Advanced	话中带刺
0494	Newbie	Newbie	Quiet for the Baby
0493	Intermediate	Intermediate	Subway Announcements
0492	Elementary	Elementary	Shut up!
0491	Upper Intermediate	Upper Intermediate	Chinese Onomatopoeia
0490	Newbie	Newbie	Calling Roll
0489	Advanced	Advanced	自然灾害
0488	Intermediate	Intermediate	A Business Dinner with a Supplier
0487	Elementary	Elementary	The Clogged Toilet
0486	Intermediate	Intermediate	Lili and Zhang Liang 12: The Breakup
0485	Media	Media	废除非法同居
0484	Newbie	Newbie	Head, Shoulders, Knees, and Toes
0483	Upper Intermediate	Upper Intermediate	Chinese Geography
0482	Elementary	Elementary	Flags and National Anthems
0481	Advanced	Advanced	接触爵士
0480	Newbie	Newbie	Wireless Internet
0479	Upper Intermediate	Upper Intermediate	State-Owned

0304	Newbie	Newbie	Playing Cards
0302	Intermediate	Intermediate	Parent-Teacher Meeting
0303	Advanced	Advanced	上海的典型建筑
0301	Newbie	Newbie	The Door
0299	Elementary	Elementary	Elevator Emergency
0300	Media	Media	富豪征婚网
0298	Upper Intermediate	Upper Intermediate	Movie Genres
0296	Newbie	Newbie	Asking for Change
0297	Advanced	Advanced	崇洋媚外
0295	Elementary	Elementary	Just a Flesh Wound
0293	Intermediate	Intermediate	Lili and Zhang Liang 3: The Jealous Friend
0294	Advanced	Advanced	中西方简历的不同
0292	Elementary	Elementary	Introducing One's Children
0290	Newbie	Newbie	Stopping the Taxi
0291	Media	Media	鸿富锦诉讼
0289	Upper Intermediate	Upper Intermediate	Chinese Antiques
0287	Elementary	Elementary	Beijing Olympics: Opening Ceremony
0288	Advanced	Advanced	八十年代经典卡通
0286	Newbie	Newbie	I'm cold!
0284	Intermediate	Intermediate	Adoption
0285	Advanced	Advanced	传媒大战
0283	Elementary	Elementary	Weather and Seasons
0281	Newbie	Newbie	Feminine Products
0282	Media	Media	太阳系只剩8大行星
0280	Upper Intermediate	Upper Interm

0106	Intermediate	Intermediate	How was Your Date?
0105	Elementary	Elementary	What would you like to drink?
0104	Intermediate	Intermediate	Future Goals
0103	Newbie	Newbie	Buying Train Tickets
0102	Intermediate	Intermediate	A Disappointment
0101	Elementary	Elementary	My boss isn't satisfied
0100	Intermediate	Intermediate	Negotiating Price
0099	Newbie	Newbie	Asking for Favors
0098	Elementary	Elementary	Connecting with Email
0097	Newbie	Newbie	Invitations to Eat (original)
0096	Pre Intermediate	Pre Intermediate	Buying a Computer
0095	Newbie	Newbie	Addresses
0094	Newbie	Newbie	Sending Emails
0093	Elementary	Elementary	Office Policy: Internet
0092	Newbie	Newbie	Surfing the Internet (original)
0091	Elementary	Elementary	Mother's  Birthday
0090	Newbie	Newbie	University Major: Part 1
0089	Elementary	Elementary	Football (Soccer)
0088	Newbie	Newbie	Marriage and Divorce (original)
0087	Elementary	Elementary	Hobbies: Music
0086	Intermediate	Intermediate	Apartment Hunting
0085	Intermediate	Intermedi

In [31]:
cp_lesson_df = pd.DataFrame(cp_lesson_dics)

In [32]:
cp_lesson_df.head()

,Key,Name,Level,Series,URL,Teachers,Date,Video,Tags,Functions,ImageURL,Description
0,QW0465,Five ways of seeing: 看、见、看见、看到、看出,Pre-Intermediate,Qing Wen,https://chinesepod.com/lesson-redirect?v3id=QW...,"[Joy, Joe]","January 10, 2020",True,"[video, synonyms]",[],https://s3contents.chinesepod.com/extra/QW0465...,"11:19 \nIn this Qingwen, we answer a quest..."
1,4263,Animal Proverbs: Become a pig to catch a tiger,Intermediate,Intermediate,https://chinesepod.com/lesson-redirect?v3id=4263,"[Joy, Joshua]","January 8, 2020",True,"[video, animal proverbs, animals]",[],https://s3contents.chinesepod.com/4263/400906d...,10:07 \n\nAnimal metaphors are prevalent i...
2,4423,相见恨晚——老电影,Upper Intermediate,Upper Intermediate,https://chinesepod.com/lesson-redirect?v3id=4423,"[Mikkelson, Peony]","January 6, 2020",False,"[movies, film]",[],https://s3contents.chinesepod.com/4423/7bbf9e1...,“有心栽花花不开，无心插柳柳成荫。”费尽心思种下的花朵迟迟不开，随手插下的柳枝却长成一片树荫...
3,4272,Character mind map: 客 guest,Elementary,Elementary,https://chinesepod.com/lesson-redirect?v3id=4272,"[Joy, Joe]","January 3, 2020",True,"[video, Character Mind Map, character]",[],https://s3contents.chinesepod.com/4272/733551a...,"9:15 \n\n\nIn Chinese, you often encounter..."
4,4271,How To Use Action Complete “le” 了,Pre Intermediate,Pre Intermediate,https://chinesepod.com/lesson-redirect?v3id=4271,"[Joshua, Jessica]","December 31, 2019",True,"[video, grammar, le, actions]",[],https://s3contents.chinesepod.com/4271/7665f59...,"10:28 \nIn this lesson, we're learning the..."


In [33]:
len(cp_lesson_df)

4305

## Store Data

In [34]:
def store_cp_data(data,fileName,directory):
    
    """
        Stores the ChinesePod data stored in the pandas dataframe 'data' as fileName.h5 in
        directory.
        
    """
    
    # Create directory if doesn't already exist
    if not os.path.exists(directory):
        os.mkdir(directory)
        print("Directory: " , directory ,  " created.")
    else:    
        print("Directory: " , directory ,  " already exists.")   
    
    # Extension
    fileName += '.h5'
    
    # Path and name
    fileName = os.path.join(directory,fileName)

    print('Data to be stored in ', fileName)
    
    store = pd.HDFStore(fileName)
    store['data'] = data
    store.close()

In [35]:
store_cp_data(data=cp_lesson_df,fileName='ChinesePod-Library',directory=local_CP_dir)

Directory:  ./data  already exists.
Data to be stored in  ./data/ChinesePod-Library.h5


/Users/lappy/anaconda3/envs/webscraping_env37/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['Key', 'Name', 'Level', 'Series', 'URL', 'Teachers', 'Date', 'Tags', 'Functions', 'ImageURL', 'Description']]

  """Entry point for launching an IPython kernel.
